In [1]:
import tensorflow as tf
import numpy as np
import sonnet as snt
# set Ture during training, False during testing
TRAINING = True

def gaussian_mask(u, s, d, R, C):
    """
    :param u: tf.Tensor, centre of the first Gaussian.
    :param s: tf.Tensor, standard deviation of Gaussians.
    :param d: tf.Tensor, shift between Gaussian centres.
    :param R: int, number of rows in the mask, there is one Gaussian per row.
    :param C: int, number of columns in the mask.
    """
    # indices to create centres
    R = tf.to_float(tf.reshape(tf.range(R), (1, 1, R)))
    print('R',np.shape(R))
    C = tf.to_float(tf.reshape(tf.range(C), (1, C, 1)))
    print('C',np.shape(C))
    centres = u[np.newaxis, :, np.newaxis] + R * d
    print('centres',np.shape(centres))
    column_centres = C - centres
    print('column_centres',np.shape(column_centres))
    mask = tf.exp(-.5 * tf.square(column_centres / s))
    print('mask',np.shape(mask))
    # we add eps for numerical stability
    normalised_mask = mask / (tf.reduce_sum(mask, 1, keep_dims=True) + 1e-8)
    print('normalised_mask',np.shape(normalised_mask))
    batch_normalised_mask=[]
    for _ in range(64):
        batch_normalised_mask.append(normalised_mask[0])
    return batch_normalised_mask

def gaussian_glimpse(img_tensor, transform_params, crop_size):
    """
    :param img_tensor: tf.Tensor of size (batch_size, Height, Width, channels)
    :param transform_params: tf.Tensor of size (batch_size, 6), where params are  (mean_y, std_y, d_y, mean_x, std_x, d_x) specified in pixels.
    :param crop_size): tuple of 2 ints, size of the resulting crop
    """
    # parse arguments
    h, w = crop_size
    H, W = img_tensor.shape.as_list()[1:3]
    split_ax = transform_params.shape.ndims -1
    uy, sy, dy, ux, sx, dx = tf.split(transform_params, 6, split_ax)
    # create Gaussian masks, one for each axis
    Ay = gaussian_mask(uy, sy, dy, h, H)
    print('Ay',np.shape(Ay))
    Ax = gaussian_mask(ux, sx, dx, w, W)
    print('Ax',np.shape(Ax))
    # extract glimpse
    glimpse = tf.matmul(tf.matmul(img_tensor,Ay, adjoint_a=True), Ax)
    return glimpse

def spatial_transformer(img_tensor, transform_params, crop_size):
    """
    :param img_tensor: tf.Tensor of size (batch_size, Height, Width, channels)
    :param transform_params: tf.Tensor of size (batch_size, 4), where params are  (scale_y, shift_y, scale_x, shift_x)
    :param crop_size): tuple of 2 ints, size of the resulting crop
    """
    constraints = snt.AffineWarpConstraints.no_shear_2d()
    print('constraints',np.shape(constraints))
    img_size = img_tensor.shape.as_list()[1:]
    print('img_size',img_size)
    warper = snt.AffineGridWarper(img_size, crop_size, constraints)
    grid_coords = warper(transform_params)
    glimpse = tf.contrib.resampler.resampler(img_tensor[..., tf.newaxis], grid_coords)
    return glimpse


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [2]:
def identity_block(X_input, kernel_size, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    with tf.name_scope("id_block_stage"+str(stage)):
        filter1, filter2, filter3 = filters
        X_shortcut = X_input

        # First component of main path
        x = tf.layers.conv2d(X_input, filter1,
                 kernel_size=(1, 1), strides=(1, 1),name=conv_name_base+'2a')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base+'2a', training=TRAINING)
        x = tf.nn.relu(x)

        # Second component of main path
        x = tf.layers.conv2d(x, filter2, (kernel_size, kernel_size),
                                 padding='same', name=conv_name_base+'2b')
        # batch_norm2 = tf.layers.batch_normalization(conv2, axis=3, name=bn_name_base+'2b', training=TRAINING)
        x = tf.nn.relu(x)

        # Third component of main path
        x = tf.layers.conv2d(x, filter3, kernel_size=(1, 1),name=conv_name_base+'2c')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base + '2c', training=TRAINING)

        # Final step: Add shortcut value to main path, and pass it through a RELU activation
        X_add_shortcut = tf.add(x, X_shortcut)
        add_result = tf.nn.relu(X_add_shortcut)

    return add_result


def convolutional_block(X_input, kernel_size, filters, stage, block, stride = 2):
    #change the shape of output so that it can do sum process with shotcut
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    with tf.name_scope("conv_block_stage" + str(stage)):

        # Retrieve Filters
        filter1, filter2, filter3 = filters

        # Save the input value
        X_shortcut = X_input

        # First component of main path
        x = tf.layers.conv2d(X_input, filter1,
                                 kernel_size=(1, 1),
                                 strides=(stride, stride),
                                 name=conv_name_base+'2a')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base+'2a', training=TRAINING)
        x = tf.nn.relu(x)

        # Second component of main path
        x = tf.layers.conv2d(x, filter2, (kernel_size, kernel_size), name=conv_name_base + '2b',padding='same')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base + '2b', training=TRAINING)
        x = tf.nn.relu(x)

        # Third component of main path
        x = tf.layers.conv2d(x, filter3, (1, 1), name=conv_name_base + '2c')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base + '2c', training=TRAINING)

        # SHORTCUT PATH
        X_shortcut = tf.layers.conv2d(X_shortcut, filter3, (1,1),
                                      strides=(stride, stride), name=conv_name_base + '1')
        X_shortcut = tf.layers.batch_normalization(X_shortcut, axis=3, name=bn_name_base + '1', training=TRAINING)

        # Final step: Add shortcut value to main path, and pass it through a RELU activation
        X_add_shortcut = tf.add(X_shortcut, x)
        add_result = tf.nn.relu(X_add_shortcut)

    return add_result



after_attention = []
def ResNet50_reference(X, classes= 2):
#     x = tf.pad(X, tf.constant([[0, 0],[3, 3,], [3, 3], [0, 0]]), "CONSTANT")

#     assert(x.shape == (x.shape[0], 70, 70, 1))
    global after_attention
    
    print('X',np.shape(X))
    img_x, img_y=np.shape(X)[1:]
    glimpse_size = 64,64
#     u = 2.
#     s = .5
#     d = 1.
#     u, s, d = (np.asarray([i]) for i in (u, s, d))
#     para=[]
#     for _ in range(64):
#         para.append([2.,.5,1.,2.,.5,1.])
#     print(np.shape(para))
#     gaussian_att_params = tf.cast(tf.reshape(para, [64,6]),tf.float32)
#     gaussian_att_params = np.array(para)
   
    transform = [.4, -.1, .4, -.1]
    para_spt=[]
    for _ in range(64):
        para_spt.append([.4, -.1, .4, -.1])
    print('para_spt',np.shape(para_spt)) 
    para_spt=tf.cast(para_spt,tf.float32)
    
    # attention
    x = spatial_transformer(tf.reshape(X,[64,img_x,img_y]), para_spt, glimpse_size)
    after_attention.append(x)
    # stage 1
    x = tf.layers.conv2d(x, filters=64, kernel_size=(3, 3), strides=(2, 2), name='conv1')
    x = tf.layers.batch_normalization(x, axis=3, name='bn_conv1')
    x = tf.nn.relu(x)
    # local response normalization
    x = tf.nn.lrn(x, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')
    x = tf.layers.max_pooling2d(x, pool_size=(3, 3),strides=(2, 2))
    
#      x = convolutional_block(x, kernel_size=3, filters=[64, 64, 64], stage=2, block='attention_a', stride=1)
#     x = identity_block(x, 3, [64, 64, 64], stage=0, block='attention_b')
#     x = identity_block(x, 3, [64, 64, 64], stage=0, block='attention_c')
#     x = gaussian_glimpse(x, gaussian_att_params, glimpse_size)
    
    # stage 2
    x = convolutional_block(x, kernel_size=3, filters=[64, 64, 256], stage=2, block='a', stride=1)
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
    
#     # attention
#     x = convolutional_block(x, kernel_size=3, filters=[64, 64, 64], stage=2, block='attention_a', stride=1)
#     x = identity_block(x, 3, [64, 64, 64], stage=0, block='attention_b')
#     x = identity_block(x, 3, [64, 64, 64], stage=0, block='attention_c')
#     x = gaussian_glimpse(x, gaussian_att_params, glimpse_size)

    # stage 3
    x = convolutional_block(x, kernel_size=3, filters=[128,128,512],
                                            stage=3, block='a', stride=2)
    x = identity_block(x, 3, [128,128,512], stage=3, block='b')
    x = identity_block(x, 3, [128,128,512], stage=3, block='c')
    x = identity_block(x, 3, [128,128,512], stage=3, block='d')

    # stage 4
    x = convolutional_block(x, kernel_size=3, filters=[256, 256, 1024], stage=4, block='a', stride=2)
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    # stage 5
    x = convolutional_block(x,kernel_size=3,filters=[512, 512, 2048], stage=5, block='a', stride=2)
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    #x = tf.layers.average_pooling2d(x, pool_size=(2, 2), strides=(1,1))
    # local response normalization
    x = tf.nn.lrn(x, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
    x = tf.layers.max_pooling2d(x, pool_size=(2, 2), strides=(1,1))

    flatten = tf.layers.flatten(x, name='flatten')
    keep_prob = 0.5
    # dropout
    drop = tf.nn.dropout(flatten, keep_prob)
    dense1 = tf.layers.dense(drop, units=50, activation=tf.nn.relu)
    logits = tf.layers.dense(dense1, units=2, activation=tf.nn.softmax)
    return logits


In [3]:
def losses(logits, labels, name):
    with tf.variable_scope('loss') as scope:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits \
            (logits=logits, labels=labels, name='xentropy_per_example')
        loss = tf.reduce_mean(cross_entropy, name='loss')
        if name=='train':
            tf.summary.scalar(scope.name + '/train_loss', loss)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_loss', loss)
    return loss
 
def trainning(loss, learning_rate):
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step= global_step)
    return train_op
 
def evaluation(logits, labels, name):
    with tf.variable_scope('accuracy') as scope:
        correct = tf.nn.in_top_k(logits, labels, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_accuracy', accuracy)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_accuracy', accuracy)
    return accuracy

def recall_precision(logits, labels, name):
    logits = tf.cast(logits, tf.int64)
    labels = tf.cast(labels, tf.int64)
    predict = tf.arg_max(logits,1)
    with tf.variable_scope('recall_precision') as scope:
        TP = tf.count_nonzero(predict * labels)
        TN = tf.count_nonzero((predict - 1) * (labels - 1))
        FN = tf.count_nonzero(predict * (labels - 1))
        FP = tf.count_nonzero((predict - 1) * labels)
        precision = tf.divide(TP, TP + FP)
        recall = tf.divide(TP, TP + FN)
        precision = tf.cast(precision, dtype=tf.float64)
        recall = tf.cast(recall, dtype=tf.float64)
        #f1 = 2 * precision * recall / (precision + recall)
        #f1 = tf.cast(f1, dtype=tf.float32)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_precision', precision)
            tf.summary.scalar(scope.name + '/train_recall', recall)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_precision', precision)
            tf.summary.scalar(scope.name + '/valid_recall', recall)
    return precision, recall

In [4]:
import os
 
def get_files(path_pos,path_neg,label_pos,label_neg):
    TC = []
    label_TC = []
    nonTC = []
    label_nonTC = []
    # data loader
    file_dir_TC=path_pos
    file_dir_nonTC=path_neg
    TC_list = os.listdir(file_dir_TC)
    nonTC_list = os.listdir(file_dir_nonTC)
    for file in TC_list[:len(nonTC_list)]:
        name = file.split('_')
        if name[0] == label_pos:
            TC.append(file_dir_TC + file)
            label_TC.append(1)
    for file in nonTC_list:
        name = file.split('_')
        if name[0] == label_neg:
            nonTC.append(file_dir_nonTC + file)
            label_nonTC.append(0)
    print("There are %d TC\nThere are %d nonTC" % (len(TC), len(nonTC)))
 
    # shuffle
    image_list = np.hstack((TC, nonTC))
    label_list = np.hstack((label_TC, label_nonTC))
    temp = np.array([image_list, label_list])
    temp = temp.transpose()    
    np.random.shuffle(temp)
 
    image_list = list(temp[:, 0])
    label_list = list(temp[:, 1])
    label_list = [int(i) for i in label_list]
 
    return image_list, label_list
 
# img_list,label_list = get_files(file_dir)
 
# batch
def get_batch(image, label, image_W, image_H, batch_size, capacity):   
    image = tf.cast(image, tf.string)
    label = tf.cast(label, tf.int32)
 
    # queue
    input_queue = tf.train.slice_input_producer([image, label])
 
    image_contents = tf.read_file(input_queue[0])
    label = input_queue[1]
    image = tf.image.decode_jpeg(image_contents, channels=1)
 
    # resize
    image = tf.image.resize_images(image, [image_H, image_W], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = tf.cast(image, tf.float32)
    # image = tf.image.per_image_standardization(image)  
    image_batch, label_batch = tf.train.batch([image, label],
                                              batch_size=batch_size,
                                              num_threads=64,  
                                              capacity=capacity)
  
    return tf.reshape(image_batch,[batch_size,image_W,image_H]), label_batch

In [ ]:
import os
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
N_CLASSES = 2 
IMG_W = 64  # resize
IMG_H = 64
BATCH_SIZE = 64
CAPACITY = 2000
MAX_STEP = 20000000 
learning_rate = 0.0001 
 
train_dir = './train_attention_resnet_git/'
logs_train_dir = './train_attention_resnet_git/'

file_dir_TC='/home/ubuntu/data/TC/'
file_dir_nonTC='/home/ubuntu/data/nonTC/'
file_dir_valTC='/home/ubuntu/data/valTC_cp/'
file_dir_valnonTC='/home/ubuntu/data/valnonTC/'

train, train_label = get_files(file_dir_TC,file_dir_nonTC,'TC','nonTC')
valid, valid_label = get_files(file_dir_valTC,file_dir_valnonTC,'valTC','valnonTC')
train_batch_op,train_label_batch_op=get_batch(train,
                                train_label,
                                IMG_W,
                                IMG_H,
                                BATCH_SIZE,
                                CAPACITY)

valid_batch_op,valid_label_batch_op=get_batch(valid,
                                valid_label,
                                IMG_W,
                                IMG_H,
                                BATCH_SIZE,
                                CAPACITY)
x = tf.placeholder(tf.float32, [BATCH_SIZE,IMG_W,IMG_H])
y = tf.placeholder(tf.float32, [BATCH_SIZE])
y = tf.cast(y,tf.int64)

train_logits = ResNet50_reference(x)
constant=[]
for _ in range(BATCH_SIZE):
    constant.append([0.5,0.])
    
valid_logits_penalty = tf.add(train_logits,constant)
train_loss = losses(train_logits, y, 'train')
train_op = trainning(train_loss, learning_rate)
train_acc = evaluation(train_logits, y, 'train')
train_recall, train_precision = recall_precision(train_logits, y, 'train')

valid_loss_penalty = losses(valid_logits_penalty, y, 'valid')
valid_loss = losses(valid_logits_penalty, y, 'valid')
valid_acc = evaluation(train_logits, y, 'valid')
valid_recall, valid_precision = recall_precision(train_logits, y, 'valid')

summary_op = tf.summary.merge_all() 
 
sess = tf.Session()

train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)

saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())
 

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
 
try:
    print('----------training start---------')
    train_result=[]
    valid_result=[]
    for step in np.arange(MAX_STEP):
        train_batch,train_label_batch = sess.run([train_batch_op,train_label_batch_op])
        if coord.should_stop():
                break
       
        _, tra_loss, tra_acc, tra_recall, tra_precision= sess.run([train_op, train_loss, train_acc, train_recall, train_precision],feed_dict={x:train_batch,y:train_label_batch})
        summary_str = sess.run(summary_op,feed_dict={x:train_batch,y:train_label_batch})
        train_writer.add_summary(summary_str, step)
        train_result.append([step, tra_loss, tra_acc*100.0, tra_recall*100.0, tra_precision*100.0])
        if step % 50 == 0:
            print('Step %d, train loss = %.2f, train accuracy = %.2f%%, train recall = %.2f%%, train precision = %.2f%%' %(step, tra_loss, tra_acc*100.0, tra_recall*100.0, tra_precision*100.0))
            
            summary = tf.Summary()
            summary.value.add(tag='tra_loss', simple_value=tra_loss)
            summary.value.add(tag='tra_acc', simple_value=tra_acc)
            summary.value.add(tag='tra_recall', simple_value=tra_recall)
            summary.value.add(tag='tra_precision', simple_value=tra_precision)
            train_writer.add_summary(summary, step)
        if step % 500 == 0:
            ckpt=tf.train.get_checkpoint_state('./train_attention_resnet_git/')
#             print(ckpt)
            if ckpt and ckpt.all_model_checkpoint_paths:
                valid_batch,valid_label_batch = sess.run([valid_batch_op,valid_label_batch_op])
                saver.restore(sess,ckpt.model_checkpoint_path)
                val_loss, val_loss_penalty, val_acc, val_recall, val_precision = sess.run([valid_loss, valid_loss_penalty, valid_acc, valid_recall, valid_precision], feed_dict={x:valid_batch,y:valid_label_batch})
                valid_result.append([step, val_loss, val_loss_penalty, val_acc*100.0, val_recall*100.0, val_precision*100.0])
                print('*********************')
                print('Step %d, valid loss = %.2f, valid accuracy = %.2f%%, valid recall = %.2f%%, valid precision = %.2f%%' %(step, val_loss, val_acc*100.0, val_recall*100.0, val_precision*100.0))
                print('*********************')
                summary = tf.Summary()
                summary.value.add(tag='val_loss', simple_value=val_loss)
                summary.value.add(tag='val_loss_penalty', simple_value=val_loss_penalty)
                summary.value.add(tag='val_acc', simple_value=val_acc)
                summary.value.add(tag='val_recall', simple_value=val_recall)
                summary.value.add(tag='val_precision', simple_value=val_precision)
                train_writer.add_summary(summary, step)
                summary.value.add(tag='recall_precision_curve', simple_value=val_precision)
                train_writer.add_summary(summary, val_recall)
#                 summary_str = sess.run(summary_op,feed_dict={x:valid_batch,y:valid_label_batch})
#                 train_writer.add_summary(summary_str, step)
            
      
        if step % 500 == 0 or (step + 1) == MAX_STEP:
            checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=step)

except tf.errors.OutOfRangeError:
    print('Done training -- epoch limit reached')

finally:
    coord.request_stop()

There are 1737956 TC
There are 1737956 nonTC
There are 57420 TC
There are 434488 nonTC
X (64, 64, 64)
para_spt (64, 4)
constraints ()
img_size [64, 64]
----------training start---------
Step 0, train loss = 0.79, train accuracy = 48.44%, train recall = 3.70%, train precision = 50.00%
Step 50, train loss = 0.58, train accuracy = 71.88%, train recall = 17.86%, train precision = 83.33%
Step 100, train loss = 0.65, train accuracy = 65.62%, train recall = 48.00%, train precision = 92.31%
Step 150, train loss = 0.54, train accuracy = 76.56%, train recall = 50.00%, train precision = 94.44%
Step 200, train loss = 0.52, train accuracy = 78.12%, train recall = 40.00%, train precision = 85.71%
Step 250, train loss = 0.63, train accuracy = 68.75%, train recall = 56.25%, train precision = 78.26%
Step 300, train loss = 0.49, train accuracy = 82.81%, train recall = 79.31%, train precision = 85.19%
Step 350, train loss = 0.48, train accuracy = 82.81%, train recall = 76.67%, train precision = 88.46%
St

Step 3250, train loss = 0.59, train accuracy = 71.88%, train recall = 56.76%, train precision = 91.30%
Step 3300, train loss = 0.48, train accuracy = 82.81%, train recall = 72.73%, train precision = 96.00%
Step 3350, train loss = 0.52, train accuracy = 79.69%, train recall = 65.79%, train precision = 96.15%
Step 3400, train loss = 0.50, train accuracy = 81.25%, train recall = 60.00%, train precision = 85.71%
Step 3450, train loss = 0.53, train accuracy = 78.12%, train recall = 62.50%, train precision = 86.96%
Step 3500, train loss = 0.55, train accuracy = 76.56%, train recall = 67.86%, train precision = 73.08%
INFO:tensorflow:Restoring parameters from ./train_attention_resnet_git/model.ckpt-3000
*********************
Step 3500, valid loss = 0.66, valid accuracy = 42.19%, valid recall = 0.00%, valid precision = nan%
*********************
Step 3550, train loss = 0.56, train accuracy = 75.00%, train recall = 16.67%, train precision = 100.00%
Step 3600, train loss = 0.54, train accuracy = 

*********************
Step 6500, valid loss = 0.68, valid accuracy = 39.06%, valid recall = 0.00%, valid precision = 0.00%
*********************
Step 6550, train loss = 0.58, train accuracy = 71.88%, train recall = 10.34%, train precision = 100.00%
Step 6600, train loss = 0.50, train accuracy = 81.25%, train recall = 40.00%, train precision = 85.71%
Step 6650, train loss = 0.54, train accuracy = 76.56%, train recall = 55.56%, train precision = 83.33%
Step 6700, train loss = 0.47, train accuracy = 84.38%, train recall = 57.14%, train precision = 95.24%
Step 6750, train loss = 0.54, train accuracy = 76.56%, train recall = 60.61%, train precision = 83.33%
Step 6800, train loss = 0.52, train accuracy = 79.69%, train recall = 63.64%, train precision = 100.00%
Step 6850, train loss = 0.44, train accuracy = 87.50%, train recall = 76.47%, train precision = 100.00%
Step 6900, train loss = 0.57, train accuracy = 75.00%, train recall = 75.00%, train precision = 75.00%
Step 6950, train loss = 0.51

Step 9800, train loss = 0.51, train accuracy = 79.69%, train recall = 50.00%, train precision = 94.44%
Step 9850, train loss = 0.47, train accuracy = 84.38%, train recall = 76.92%, train precision = 86.96%
Step 9900, train loss = 0.53, train accuracy = 76.56%, train recall = 60.00%, train precision = 81.82%
Step 9950, train loss = 0.56, train accuracy = 75.00%, train recall = 63.33%, train precision = 79.17%
Step 10000, train loss = 0.56, train accuracy = 75.00%, train recall = 61.76%, train precision = 80.77%
INFO:tensorflow:Restoring parameters from ./train_attention_resnet_git/model.ckpt-9500
*********************
Step 10000, valid loss = 0.69, valid accuracy = 32.81%, valid recall = 0.00%, valid precision = 0.00%
*********************
Step 10050, train loss = 0.52, train accuracy = 78.12%, train recall = 23.08%, train precision = 100.00%
Step 10100, train loss = 0.59, train accuracy = 71.88%, train recall = 47.50%, train precision = 95.00%
Step 10150, train loss = 0.55, train accur

*********************
Step 13000, valid loss = 0.72, valid accuracy = 39.06%, valid recall = 7.69%, valid precision = 100.00%
*********************
Step 13050, train loss = 0.63, train accuracy = 68.75%, train recall = 40.00%, train precision = 100.00%
Step 13100, train loss = 0.48, train accuracy = 84.38%, train recall = 22.58%, train precision = 87.50%
Step 13150, train loss = 0.39, train accuracy = 92.19%, train recall = 70.97%, train precision = 100.00%
Step 13200, train loss = 0.53, train accuracy = 78.12%, train recall = 64.29%, train precision = 94.74%
Step 13250, train loss = 0.68, train accuracy = 62.50%, train recall = 51.61%, train precision = 84.21%
Step 13300, train loss = 0.54, train accuracy = 78.12%, train recall = 73.68%, train precision = 60.87%
Step 13350, train loss = 0.44, train accuracy = 85.94%, train recall = 72.73%, train precision = 96.00%
Step 13400, train loss = 0.60, train accuracy = 70.31%, train recall = 51.35%, train precision = 90.48%
Step 13450, train 

...

Step 42250, train loss = 0.54, train accuracy = 78.12%, train recall = 60.61%, train precision = 86.96%
Step 42300, train loss = 0.57, train accuracy = 75.00%, train recall = 64.00%, train precision = 66.67%
Step 42350, train loss = 0.54, train accuracy = 76.56%, train recall = 57.58%, train precision = 90.48%
Step 42400, train loss = 0.55, train accuracy = 76.56%, train recall = 62.16%, train precision = 92.00%
Step 42450, train loss = 0.52, train accuracy = 79.69%, train recall = 60.00%, train precision = 88.89%
Step 42500, train loss = 0.50, train accuracy = 81.25%, train recall = 66.67%, train precision = 95.65%
INFO:tensorflow:Restoring parameters from ./train_attention_resnet_git/model.ckpt-42000
*********************
Step 42500, valid loss = 0.62, valid accuracy = 50.00%, valid recall = 0.00%, valid precision = nan%
*********************
Step 42550, train loss = 0.53, train accuracy = 78.12%, train recall = 0.00%, train precision = nan%
Step 42600, train loss = 0.55, train accur

*********************
Step 45500, valid loss = 0.55, valid accuracy = 56.25%, valid recall = 0.00%, valid precision = nan%
*********************
Step 45550, train loss = 0.65, train accuracy = 64.06%, train recall = 21.43%, train precision = 85.71%
Step 45600, train loss = 0.55, train accuracy = 76.56%, train recall = 25.00%, train precision = 100.00%
Step 45650, train loss = 0.51, train accuracy = 79.69%, train recall = 58.62%, train precision = 89.47%
Step 45700, train loss = 0.51, train accuracy = 79.69%, train recall = 53.57%, train precision = 83.33%
Step 45750, train loss = 0.51, train accuracy = 79.69%, train recall = 69.57%, train precision = 69.57%
Step 45800, train loss = 0.63, train accuracy = 67.19%, train recall = 63.33%, train precision = 70.37%
Step 45850, train loss = 0.53, train accuracy = 78.12%, train recall = 82.14%, train precision = 71.88%
Step 45900, train loss = 0.55, train accuracy = 76.56%, train recall = 58.06%, train precision = 94.74%
Step 45950, train loss

Step 48750, train loss = 0.53, train accuracy = 78.12%, train recall = 59.26%, train precision = 100.00%
Step 48800, train loss = 0.58, train accuracy = 73.44%, train recall = 54.84%, train precision = 89.47%
Step 48850, train loss = 0.41, train accuracy = 90.62%, train recall = 78.12%, train precision = 96.15%
Step 48900, train loss = 0.46, train accuracy = 85.94%, train recall = 66.67%, train precision = 95.24%
Step 48950, train loss = 0.51, train accuracy = 79.69%, train recall = 73.08%, train precision = 79.17%
Step 49000, train loss = 0.45, train accuracy = 85.94%, train recall = 66.67%, train precision = 96.00%
INFO:tensorflow:Restoring parameters from ./train_attention_resnet_git/model.ckpt-48500
*********************
Step 49000, valid loss = 0.69, valid accuracy = 45.31%, valid recall = 0.00%, valid precision = 0.00%
*********************
Step 49050, train loss = 0.55, train accuracy = 76.56%, train recall = 25.93%, train precision = 70.00%
Step 49100, train loss = 0.60, train 

*********************
Step 52000, valid loss = 0.70, valid accuracy = 42.19%, valid recall = 0.00%, valid precision = nan%
*********************
Step 52050, train loss = 0.63, train accuracy = 67.19%, train recall = 10.81%, train precision = 80.00%
Step 52100, train loss = 0.56, train accuracy = 75.00%, train recall = 32.35%, train precision = 84.62%
Step 52150, train loss = 0.62, train accuracy = 68.75%, train recall = 41.18%, train precision = 77.78%
Step 52200, train loss = 0.53, train accuracy = 76.56%, train recall = 51.85%, train precision = 87.50%
Step 52250, train loss = 0.61, train accuracy = 70.31%, train recall = 39.39%, train precision = 100.00%
Step 52300, train loss = 0.55, train accuracy = 76.56%, train recall = 54.29%, train precision = 95.00%
Step 52350, train loss = 0.56, train accuracy = 75.00%, train recall = 60.61%, train precision = 90.91%
Step 52400, train loss = 0.52, train accuracy = 79.69%, train recall = 51.43%, train precision = 94.74%
Step 52450, train loss

Step 55250, train loss = 0.42, train accuracy = 89.06%, train recall = 77.14%, train precision = 96.43%
Step 55300, train loss = 0.49, train accuracy = 81.25%, train recall = 66.67%, train precision = 86.96%
Step 55350, train loss = 0.51, train accuracy = 79.69%, train recall = 53.12%, train precision = 94.44%
Step 55400, train loss = 0.45, train accuracy = 85.94%, train recall = 92.59%, train precision = 75.76%
Step 55450, train loss = 0.56, train accuracy = 75.00%, train recall = 76.92%, train precision = 66.67%
Step 55500, train loss = 0.53, train accuracy = 78.12%, train recall = 58.33%, train precision = 91.30%
INFO:tensorflow:Restoring parameters from ./train_attention_resnet_git/model.ckpt-55000
*********************
Step 55500, valid loss = 0.72, valid accuracy = 45.31%, valid recall = 0.00%, valid precision = nan%
*********************
Step 55550, train loss = 0.58, train accuracy = 70.31%, train recall = 31.25%, train precision = 100.00%
Step 55600, train loss = 0.46, train a